In [1]:
import ciropt as co
import cvxpy as cp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 1.

# solver = "ca_canonical_X"
solver = "gp_canonical_X"

time_limit = 100

# Ciropt problem

In [3]:
problem = co.gradient_flow_circuit(mu, L_smooth, Capacitance)

res, model, sp_exp = problem.solve(solver=solver, verbose=False, time_limit=100)[:3]
print(res)

dim_G=5, dim_F=4
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31


/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:494: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr(  (M1 @ var_x).item()  - gp_trace(M2 @ var_X) == 0)
/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:508: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  + gp_trace(M2 @ var_X) \
/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:509: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  - gp_trace(M3 @ var_X) == 0)
/Users/parshakova.tanya/Documents/projects/Circuits/circuit_analysis/ciropt/circuit_opt.py:524: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when build

{'b': 0.9999999970168865, 'h': 2.362919398175109, 'd': 1000.0, 'alpha': 0.4231896162011234, 'beta': 0.4231517686761676}


In [4]:
res

{'b': 0.9999999970168865,
 'h': 2.362919398175109,
 'd': 1000.0,
 'alpha': 0.4231896162011234,
 'beta': 0.4231517686761676}

# PEP verification

In [5]:
import PEPit
from PEPit.functions import SmoothStronglyConvexFunction, SmoothConvexFunction
from PEPit.primitive_steps import proximal_step

b = res["b"] 
h = res["h"]
alpha = res["alpha"]
beta = res["beta"]

In [6]:
problem = PEPit.PEP()
func = problem.declare_function(
                    SmoothStronglyConvexFunction,
                    mu=mu,   # Strong convexity param.
                    L=L_smooth) 


x_P_star = func.stationary_point()
g_star, f_star = func.oracle(x_P_star)
x_P_1 = problem.set_initial_point()

g_1, f_1 = func.oracle(x_P_1)
x_P_1p5 = x_P_1 - (alpha * h / Capacitance) * g_1 
g_1p5, f_1p5 = func.oracle(x_P_1p5)
x_P_2 = x_P_1  - (beta * h / Capacitance) * g_1 - ((1 - beta) * h / Capacitance) * g_1p5

g_2, f_2 = func.oracle(x_P_2)

# Set the performance metric to the function values accuracy
E_1 = (Capacitance/2) * (x_P_1 - x_P_star)**2
E_2 = (Capacitance/2) * (x_P_2 - x_P_star)**2
Delta_1 = b * (f_1 - f_star)
problem.set_performance_metric(E_2 - (E_1 - Delta_1))

# Solve the PEP with verbose = 1
verbose = 0
diff = problem.solve(verbose=verbose, solver=cp.MOSEK)
print(f"{diff=}")

diff=-1.3067190662852113e-09
